# 10. Use transfer learning for large image classification, going through these steps:

## a. Create a training set containing at least 100 images per class. For example, you could classify your own pictures based on the location (beach, mountain, city, etc.), or alternatively you can use an existing dataset (e.g., from TensorFlow Datasets).


In [4]:
# # Loading CIFAR!0

# ## Import
# from tensorflow import keras

# ## Loading data
# (training_X, training_y), (X_test, y_test) = keras.datasets.cifar10.load_data()

# ## Viewing the dimensions of training and test
# print(f"train features: {training_X.shape}")
# print(f"train target: {training_y.shape}")
# print(f"test features: {X_test.shape}")
# print(f"test target: {y_test.shape}")

In [5]:
# Imports

import tensorflow as tf
import tensorflow_datasets as tfds

In [6]:
# Loading Rock Papers Scissors

## Getting data and info of data
(test_set, valid_set, train_set), info = tfds.load(
  "rock_paper_scissors", 
  split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
  as_supervised=True, 
  with_info=True
)

## Getting the number of examples in train
dataset_size = info.splits["train"].num_examples
print(f"train size: {dataset_size}")

## Getting the different classes
class_names = info.features["label"].names
print(f"classes: {class_names}")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incomplete7QJT8V/rock_paper_scissors-train.tfrecord


  0%|          | 0/2520 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incomplete7QJT8V/rock_paper_scissors-test.tfrecord


  0%|          | 0/372 [00:00<?, ? examples/s]

Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.
train size: 2520
classes: ['rock', 'paper', 'scissors']


## b. Split it into a training set, a validation set, and a test set.

In [7]:
# # Scaling the pixels

# training_X = training_X/255.
# X_test = X_test/255.

In [8]:
# # Splitting training into train and validation (we already have test)

# ## Import
# from sklearn.model_selection import train_test_split

# ## Splitting data
# X_train, X_valid, y_train, y_valid = train_test_split(training_X, training_y, test_size=0.2, random_state=0)

Split into train, valid, and test while loading the data in previous step.

## c. Build the input pipeline, including the appropriate preprocessing operations, and optionally add data augmentation.


In [9]:
# Function that preprocesses the images

def preprocess(image, label):

  ## Xception accepts images in shape of (224, 224)
  resized_image = tf.image.resize(image, [224,224])

  ## Preprocessing the same way as during Xception
  final_image = tf.keras.applications.xception.preprocess_input(resized_image)
  
  return final_image, label

In [10]:
# Batching and prefetching

batch_size = 32

## Randomly shuffling data
train_set.shuffle(100)

## Mapping the preprocess, batching and prefetching data
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)

In [11]:
# TO DO : Add data augmentation

# Upscale images to format needed by the Transfer Model

Simply pad the images

- https://medium.com/swlh/hands-on-the-cifar-10-dataset-with-transfer-learning-2e768fd6c318
- https://www.kaggle.com/adi160/cifar-10-keras-transfer-learning


In [23]:
# upscaled = tf.keras.layers.Lambda(lambda x: tf.image.resize_image_with_pad(train_set, 160, 160))(inputs)
# upscaled

## d. Fine-tune a pretrained model on this dataset.

In [13]:
# Loading a pretrained Xception

pretrained_model = tf.keras.applications.Xception(
  include_top=False,
  weights="imagenet",
  input_tensor=None,
  input_shape=None,
  pooling=None,
  classes=3,
  classifier_activation="softmax",
)

83697664/83683744 [==============================] - 1s 0us/step


In [14]:
# Adding additional layers

## Adding Global Pooling Layer
avg = tf.keras.layers.GlobalAveragePooling2D()(pretrained_model.output)

## Adding an output as a Dense
output = tf.keras.layers.Dense(3, activation="softmax")(avg)

## Putting model together
model = tf.keras.Model(inputs=pretrained_model.input, outputs=output)

## Freezing the weights of pretrained layers
for layer in pretrained_model.layers:
  layer.trainable = False

In [15]:
# Compiling the model

model.compile(
  loss="sparse_categorical_crossentropy",
  optimizer="adam",
  metrics=["accuracy"]
)

history = model.fit(
  train_set,
  epochs=2,
  validation_data=valid_set
)

Epoch 1/2
60/60 [==============================] - 43s 174ms/step - loss: 0.6163 - accuracy: 0.8095 - val_loss: 0.3232 - val_accuracy: 0.9735
Epoch 2/2
60/60 [==============================] - 9s 142ms/step - loss: 0.2592 - accuracy: 0.9720 - val_loss: 0.1953 - val_accuracy: 0.9894


In [16]:
# Unfreezing the weights

for layer in pretrained_model.layers:
  layer.trainable = True

In [17]:
# Continuing the training 

model.compile(
  loss="sparse_categorical_crossentropy",
  optimizer="adam",
  metrics=["accuracy"]
)

history = model.fit(
  train_set,
  epochs=3,
  validation_data=valid_set
)

Epoch 1/3
60/60 [==============================] - 44s 610ms/step - loss: 0.0810 - accuracy: 0.9757 - val_loss: 0.1548 - val_accuracy: 0.9868
Epoch 2/3
60/60 [==============================] - 36s 604ms/step - loss: 0.0263 - accuracy: 0.9974 - val_loss: 3.1704 - val_accuracy: 0.6878
Epoch 3/3
60/60 [==============================] - 37s 620ms/step - loss: 0.0234 - accuracy: 0.9947 - val_loss: 19.9847 - val_accuracy: 0.3042
